# 1.4 &emsp; Simple LCA - Navigating through activities and exchanges

This tutorial, based on [Massimo's](https://github.com/massimopizzol/B4B/blob/main/02.1_Simple_LCA_selecting_activities_exchanges_revised.py) of course! will show you how to explore your database and swing through the activities and exchanges!

In [86]:
from brightway2 import *
projects.report()

[('1_1_Massimo', 0, 8.21e-05),
 ('1_2_Massimo', 1, 9.4079e-05),
 ('1_3_Massimo', 2, 0.000101769),
 ('1_4_Massimo', 1, 9.4079e-05),
 ('Tut_1_Activities_Exchanges', 0, 8.1998e-05),
 ('Tut_1_activities_exchanges', 0, 8.1998e-05),
 ('Tut_2_Projects', 0, 8.1998e-05),
 ('Tut_2_the copy', 0, 8.1998e-05),
 ('Tut_3_Databases', 0, 8.1998e-05),
 ('default', 0, 8.21e-05)]

We will be using the database created in [01_2_Massimo_Tutorial](01_2_Massimo_Tutorial.ipynb), so we are going to copy into a new project. Remember how to do it? Uncomment the following lines to copy if this is the first time you run this code. Otherwise, jump and run to `projects.set_current('1_4_Massimo')`.

In [87]:
 # projects.set_current('1_2_Massimo')

In [88]:
 # projects.copy_project('2_1_Massimo') # If you run this more than once, it will throw an error saying that 1_4_Massimo already exists!!

Even though after you copy a project, it sets you in such project, this is a sanity check, because if you run this code more thant once `copy_project` will not work!

In [89]:
projects.set_current('2_1_Massimo')

Now let's run the LCA basic analysis:

In [90]:
t_db = Database('testdb')  
functional_unit = {t_db.get('Electricity production'): 1000}
lca = LCA(functional_unit)
lca.lci()

myLCIAdata = [[('testdb', 'Carbon dioxide'), 2.0],
              [('testdb', 'Sulphur dioxide'), 2.0],
              [('testdb', 'Crude oil'), 2.0]]

method_key = ('simplemethod', 'imaginaryendpoint', 'imaginarymidpoint') 
my_method = Method(method_key)
my_method.validate(myLCIAdata) # Need to figure out what these do
my_method.register()           # Need to figure out what these do
my_method.write(myLCIAdata)    # Need to figure out what these do
my_method.load()               # Need to figure out what these do

lca = LCA(functional_unit, method_key)  # LCA calculations with method
lca.lci()
lca.lcia()
print("This is the LCA inventory")
print(lca.inventory)
print("This is the LCA characterizedinventory")
print(lca.characterized_inventory)
print('This is the environmental score:')
print(lca.score)

This is the LCA inventory
  (0, 1)	20.0
  (0, 0)	100.0
  (1, 1)	4.0
  (1, 0)	10.000000149011612
  (2, 1)	-100.0
This is the LCA characterizedinventory
  (0, 0)	200.0
  (0, 1)	40.0
  (1, 0)	20.000000298023224
  (1, 1)	8.0
  (2, 1)	-200.0
This is the environmental score:
68.00000029802322


But we already knew these things, didn't we! Let's fish some activities and exchanges:

First let's get `Electricity production`

In [29]:
el = t_db.get('Electricity production') 
print(el)

'Electricity production' (kWh, None, None)


We can check the keys of that activity dictionary:

In [30]:
for i in el:  # these the possible keys of an activity dictionary
    print(i)

name
unit
database
code


This is another way that also gives you the values:

In [37]:
el.as_dict() 

{'name': 'Electricity production',
 'unit': 'kWh',
 'database': 'testdb',
 'code': 'Electricity production'}

This option just gives you the values:

In [45]:
print(el['name'],'\n', el['code'],'\n', el['unit'],'\n', el['database'])

Electricity production 
 Electricity production 
 kWh 
 testdb


The following shows how to extract the exchanges of the activity object. Massimo here is trying to check what works and what doesn't and I find it hilarious.

In [44]:
#el['exchanges']  # this does not work.
#el.exchanges()  # neither this
list(el.exchanges())  # yeps, this one

[Exchange: 2 kg 'Fuel production' (kg, None, None) to 'Electricity production' (kWh, None, None)>,
 Exchange: 1 kg 'Carbon dioxide' (kg, None, None) to 'Electricity production' (kWh, None, None)>,
 Exchange: 0.1 kg 'Sulphur dioxide' (kg, None, None) to 'Electricity production' (kWh, None, None)>,
 Exchange: 10 kWh 'Electricity production' (kWh, None, None) to 'Electricity production' (kWh, None, None)>]

Or we can loop through them to visualize all exchanges:

In [70]:
for exc in el.exchanges():  # or this, visualize all exchanges of an activity
    print(exc,'\n', exc['type'], '\n', exc['input'][1], '\n', exc.input, '\n')

Exchange: 2 kg 'Fuel production' (kg, None, None) to 'Electricity production' (kWh, None, None)> 
 technosphere 
 Fuel production 
 'Fuel production' (kg, None, None) 

Exchange: 1 kg 'Carbon dioxide' (kg, None, None) to 'Electricity production' (kWh, None, None)> 
 biosphere 
 Carbon dioxide 
 'Carbon dioxide' (kg, None, None) 

Exchange: 0.1 kg 'Sulphur dioxide' (kg, None, None) to 'Electricity production' (kWh, None, None)> 
 biosphere 
 Sulphur dioxide 
 'Sulphur dioxide' (kg, None, None) 

Exchange: 10 kWh 'Electricity production' (kWh, None, None) to 'Electricity production' (kWh, None, None)> 
 production 
 Electricity production 
 'Electricity production' (kWh, None, None) 



To get the first exchange:

In [72]:
el_exc = list(el.exchanges())[0]

Let's compare getting three extracting method objects by looking at their `type()`:

In [74]:
print(type(el))  # compare the three
print(type(el.exchanges()))
print(type(el_exc))

<class 'bw2data.backends.peewee.proxies.Activity'>
<class 'bw2data.backends.peewee.proxies.Exchanges'>
<class 'bw2data.backends.peewee.proxies.Exchange'>


Aha! Our objects go from `Activity` to `Exchanges` (all of them) to `Exchange` (just one!)

Let's look at the things inside our exchange object:

In [75]:
for i in el_exc:  # the possible keys of an exchange (DICT iteration)
    print(i, ':', el_exc[i])

input : ('testdb', 'Fuel production')
amount : 2
unit : kg
type : technosphere
output : ('testdb', 'Electricity production')


Interesting! This one has the `output` key, but we never defined an `output` in the original database dicitonary, if something... we called it `input`, how can this be??!! This is because when an exchange is identified as `'type': 'production'`, brightway2 then knows it is an `output`!

Here are some other ways of pulling out exchanges:

In [76]:
el_exc.as_dict()

{'input': ('testdb', 'Fuel production'),
 'amount': 2,
 'unit': 'kg',
 'type': 'technosphere',
 'output': ('testdb', 'Electricity production')}

In [77]:
el_exc.items()

ItemsView(Exchange: 2 kg 'Fuel production' (kg, None, None) to 'Electricity production' (kWh, None, None)>)

In [78]:
el_exc.unit == el_exc['unit']  # equivalent ways, different from activities
print(el_exc['amount'], el_exc['unit'], el_exc['input'], 'to',
      el_exc['output'], 'within', el_exc['type'])

2 kg ('testdb', 'Fuel production') to ('testdb', 'Electricity production') within technosphere


In [81]:
el_exc.input  # I guess the intended meaning of 'input' is "from'

'Fuel production' (kg, None, None)

In [82]:
el_exc.output  # I guess the indended meaning of 'output' is 'to'

'Electricity production' (kWh, None, None)

What if I want to get a specific exchange of a specific activity. Without using numeric indexing, but by using its name! Let's see if we can find the value of the CO2 emissions from el.

In [83]:
for exc in list(el.exchanges()):
    if exc['input'] == ('testdb', 'Carbon dioxide'):
        print(exc)
    else:
        print('Not this one')

Not this one
Exchange: 1 kg 'Carbon dioxide' (kg, None, None) to 'Electricity production' (kWh, None, None)>
Not this one
Not this one


Good! Now we store the value in a variable

In [84]:
for exc in list(el.exchanges()):
    if exc['input'] == ('testdb', 'Carbon dioxide'):
        elCO2_amount = exc['amount']
        
print(elCO2_amount)

1


Ta dá!